## Problem Description

<p align="justify">An and Binh play a game consisting of multiple rounds. The first player to reach 10 points wins the game. Each round has only one winner. Assume that the rounds are independent and that An has a 0.6 probability of winning each round.</p>

<ul>
    <li>Calculate the probability that An wins the game if the winner of each round earns 2 points.</li>
    <li>Same as (a), but if the winner also won the previous round, they earn an additional 1 point.</li>
    <li>Calculate the expected number of rounds required for An to win in cases (a) and (b).</li>
</ul>

## Import neccessary python modules

In [2]:
import random # Import the random module to generate random numbers

## Solution

### `(a) Calculate the probability that An wins the game if the winner of each round earns 2 points.`

#### <div align="center"><strong>Pure math method</strong></div>
<hr>

<div align="center"><strong>Conditions for An to win</strong></div>

The game ends when a player wins **5 games** against his opponent. Therefore, An wins the game if:
1. An wins **exactly 5 games**.
2. An's **5th winning game** is the game that ends the game.
3. The game ends at **$k$**th game with $k \geq 5$, that is, An wins the $k$ game, and in the first $k-1$ games, An wins 4 games and Binh wins the remaining games.

Therefore, the number of games that the game can end in is between $k = 5, 6, 7, 8, 9$ (maximum 9 games can occur).

<div align="center"><strong>Probability expression</strong></div>

The probability that An wins the game is calculated by the sum of all possible cases:

$$
P(\text{An wins}) = \sum_{k=5}^{9} P_k
$$

Where, $ P_k $ is the probability that the game ends in the $ k $th game and An wins. This means that:
- An wins **the k th game**.
- An has won **4 games in the first $ k-1 $ games**.
- Binh wins the remaining games in the first $ k-1 $ games.

Therefore:

$$
P_k = \binom{k-1}{4} (0.6)^5 (0.4)^{k-5}
$$

So the required result is:

$$
P(\text{An thang}) = \sum_{k=5}^{9} \binom{k-1}{4} (0.6)^5 (0.4)^{k-5} \approx 0.7334323199999999
$$

#### <div align="center"><strong>Simulation method</strong></div>
<hr>

In [3]:
def simulate_game(prob_win=0.6, points_to_win=10):
    an_points, binh_points = 0, 0
    rounds = 0
    
    while an_points < points_to_win and binh_points < points_to_win:
        rounds += 1
        if random.random() < prob_win:
            an_points += 2
        else:
            binh_points += 2

    return rounds, an_points >= points_to_win

def estimate_probability(num_simulations=100000):
    wins = sum(simulate_game()[1] for _ in range(num_simulations))
    return wins / num_simulations

prob_a = estimate_probability()

print("The probability that An wins the game if the winner of each round earns 2 points:", prob_a)

The probability that An wins the game if the winner of each round earns 2 points: 0.73236


#### <div align="center"><strong>Conclusion</strong></div>
<hr>

In [4]:
print('The theoretical value is: ',0.7334323199999999)
print('The estimated value is: ',prob_a)
print('The difference between the theoretical and estimated value is: ', abs(0.7334323199999999-prob_a))

The theoretical value is:  0.7334323199999999
The estimated value is:  0.73236
The difference between the theoretical and estimated value is:  0.0010723199999999045


### `Same as (a), but if the winner also won the previous round, they earn an additional 1 point.`

#### <div align="center"><strong>Pure math method</strong></div>
<hr>

<div align="center"><strong>Problem Analysis</strong></div>

We have a game between An and Binh, with the following rules:

- If one wins consecutively, they get 3 points.

- If they don't win consecutively, they get 2 points.

- The game ends when one of them reaches 10 points.

- The probability of An winning a game is $ p $, and Binh is $ q = 1 - p $.

<div align="center"><strong>Variables used</strong></div>

Calculate the probability that An wins from state $(a, b, L)$ with:

- $ a $: An's current score.

- $ b $: Binh's current score.

- $ L $: Winner of the previous game (`None`, `'A'`, `'B'`).

<div align="center"><strong>State Table</strong></div>

We create a table $ dp[a][b][L] $, where:

- $ dp[a][b]['A'] $: Probability of An winning when the score is $ a, b $ and An won the previous game.

- $ dp[a][b]['B'] $: Probability of An winning when the score is $ a, b $ and Binh won the previous game.

- $ dp[a][b][None] $: Probability of An winning when no game has been played.

<div align="center"><strong>Basic conditions</strong></div>

- If An scores at least 10 points, the probability of winning is 1:

$$ dp[a][b][L] = 1 \quad \text{if } a \geq 10 $$
- If Binh scores at least 10 points, the probability of An winning is 0:

$$ dp[a][b][L] = 0 \quad \text{if } b \geq 10 $$

<div align="center"><strong>State transition formula</strong></div>

- **If An wins this game:**
- If An also wins the previous game ($ L = 'A' $), An receives 3 points → $ a' = a + 3 $.
- If An does not win the previous game ($ L \neq 'A' $), An receives 2 points → $ a' = a + 2 $.

- Update probability:

$$ dp[a][b][L] = p \cdot dp[a', b]['A'] $$

- **If Binh wins this game:**

- If Binh also wins the previous game ($ L = 'B' $), Binh receives 3 points → $ b' = b + 3 $.

- If Binh does not win the previous game ($ L \neq 'B' $), Binh receives 2 points → $ b' = b + 2 $.

- Update probability:

$$ dp[a][b][L] += q \cdot dp[a, b']['B'] $$

<div align="center"><strong>General formula</strong></div>

$$ dp[a][b][L] = p \cdot dp[a', b]['A'] + q \cdot dp[a, b']['B'] $$

In [5]:
# Simulation code for Dynamic Programming method

def calculate_probability(p):
    q = 1 - p
    max_score = 10
    
    dp = {L: [[0] * (max_score + 1) for _ in range(max_score + 1)] for L in [None, 'A', 'B']}

    for L in [None, 'A', 'B']:
        for b in range(max_score + 1):
            dp[L][10][b] = 1
    
        for a in range(max_score + 1):
            dp[L][a][10] = 0
    
    for a in range(max_score - 1, -1, -1):
        for b in range(max_score - 1, -1, -1):
            for L in [None, 'A', 'B']:
                a_next = a + (3 if L == 'A' else 2)
                if a_next >= max_score:
                    win_A = 1
                else:
                    win_A = dp['A'][a_next][b]

                b_next = b + (3 if L == 'B' else 2)
                if b_next >= max_score:
                    win_B = 0
                else:
                    win_B = dp['B'][a][b_next]

                dp[L][a][b] = p * win_A + q * win_B

    return dp[None][0][0]

p = 0.6
theoretical_prob_b = calculate_probability(p)

print("Probability of An winning:", theoretical_prob_b)

Probability of An winning: 0.7188341759999999


#### <div align="center"><strong>Simulation method</strong></div>
<hr>

In [6]:
def simulate_game_bonus(prob_win=0.6, points_to_win=10):
    an_points, binh_points = 0, 0
    last_winner = None
    rounds = 0

    while an_points < points_to_win and binh_points < points_to_win:
        rounds += 1
        if random.random() < prob_win:
            an_points += 2
            if last_winner == "An":
                an_points += 1
            last_winner = "An"
        else:
            binh_points += 2
            if last_winner == "Binh":
                binh_points += 1
            last_winner = "Binh"

    return rounds, an_points >= points_to_win

def estimate_probability_bonus(num_simulations=100000):
    wins = sum(simulate_game_bonus()[1] for _ in range(num_simulations))
    return wins / num_simulations

estimate_prob_b = estimate_probability_bonus()

print("Probability of An winning:", estimate_prob_b)

Probability of An winning: 0.71888


#### <div align="center"><strong>Conclusion</strong></div>
<hr>

In [7]:
print('The theoretical value is: ', theoretical_prob_b)
print('The estimated value is: ', estimate_prob_b)
print('The difference between the theoretical and estimated value is: ', abs(theoretical_prob_b - estimate_prob_b))

The theoretical value is:  0.7188341759999999
The estimated value is:  0.71888
The difference between the theoretical and estimated value is:  4.582400000008313e-05


### `Calculate the expected number of rounds required for An to win in cases (a) and (b).`

#### <div align="center"><strong>Simulation method</strong></div>
<hr>

In [9]:
def monte_carlo_simulation(num_games=100000, p=0.6, target_score=10, part='a'):
    total_rounds = 0
    an_wins = 0

    simulate_game_fn = simulate_game if part == 'a' else simulate_game_bonus

    for _ in range(num_games):
        rounds, an_win = simulate_game_fn(p, target_score)
        total_rounds += rounds
        an_wins += an_win

    probability_an_wins = an_wins / num_games
    average_rounds = total_rounds / num_games

    return probability_an_wins, average_rounds

num_games = 100000

print("Average number of games to play (a):", monte_carlo_simulation(num_games, p=0.6, target_score=10, part='a')[1])
print("Average number of games to play (b):", monte_carlo_simulation(num_games, p=0.6, target_score=10, part='b')[1])

Average number of games to play (a): 7.3509
Average number of games to play (b): 6.05303
